In [ ]:
# Department: ESTSOFT
# Class: AI Modelling
# Category: Machine learning
# Title: Data extraction
# Contributors: Jeong Gukho, Jeong Woogun, Kim Hyungeun, Kim Juneon, Kimm Soo Min
# # Last modified date: 01/04/25

### **Library**

In [2]:
# Library
import chardet
import pandas as pd
import os
import numpy as np

### **Data Extraction**

In [ ]:
def clean_dataframe(df):
    #파일 확인 해보시고 행 인덱스 위치 파악 후 삭제
    df = df.iloc[3:10]

    # 컬럼 이름 변경
    df.columns = ['년월', '시도', '시군구', '대상가구수(호)', '가구당 평균 전력 사용량(kWh)', '가구당 평균 전기요금(원)']

    # 이것도 확인 하시고 수정
    for col in ['년월', '대상가구수(호)', '가구당 평균 전력 사용량(kWh)', '가구당 평균 전기요금(원)']:
        try:
            df[col] = pd.to_numeric(df[col].str.replace(',', ''), errors='coerce')
        except AttributeError:
            pass

    return df

city=['Seoul','busan','daegu','ulsan','Daejeon','gwangju','incheon']
combined_col = ['년','월','시도','시군구','대상가구수(호)','가구당 평균 전력 사용량(kWh)','가구당 평균 전기요금(원)']

for i in city:
    folder_path = f"drive/MyDrive/{i}" #파일경로
    file_count = len(os.listdir(folder_path)) #파일 개수
    for z in range(file_count-3): #파일개수를 범위로
        if z == 0: #0일 떄는 concat 할 파일이 존재 하지않으므로 그 파일이 df_combine
            df = pd.read_excel(f"drive/MyDrive/{i}/가구 평균 월별 전력사용량_20250331.xls", engine='xlrd')
            df = clean_dataframe(df) #위에 함수 이용하여 df 청소
            df_combine = df
        else:
            df = pd.read_excel(f"drive/MyDrive/{i}/가구 평균 월별 전력사용량_20250331 ({z}).xls", engine='xlrd')
            df = clean_dataframe(df)
            df_combine = pd.concat([df_combine, df])
        combined_df=df_combine.dropna() #결측치 존재 확인 후 파악
        combined_df['년']=combined_df['년월'].astype(str).str[:4] #해당 년월에는 202401.0
        combined_df['월']=combined_df['년월'].astype(str).str[4:]
        combined_df=combined_df.drop('년월', axis=1) #년월 삭제
        combined_df=combined_df[combined_col] #colname 변경
        combined_df['월'] = combined_df['월'].astype(float).astype(int) #인트형 변경
        combined_df['년'] = combined_df['년'].astype(int) #인트형 변경
        combined_df.to_csv(f'{i}.csv', encoding='euc-kr', index=False) #인트형 변경city 이름으로 csv파일 변환

city_day=['서울','부산','대구','울산','대전','광주','인천']
city_col=['서울특별시','부산광역시','대구광역시','울산광역시','대전광역시','광주광역시','인천광역시']

num = 0
for i in city_day:
    #i를 이용하여 도시별 csv파일 불러오기
    df_overnight_heat = pd.read_csv(f"drive/MyDrive/{city[num]}/{i}_열대야.csv", encoding='cp949')
    df_heatwave = pd.read_csv(f"drive/MyDrive/{city[num]}/{i}_폭염.csv", encoding='cp949')
    df_coldwave = pd.read_csv(f"drive/MyDrive/{city[num]}/{i}_한파.csv", encoding='cp949')

    #연도로 부터 수치형 데이터가 아닌것은 뺴고 연도 데이터를 수치형 변환후 년으로 변경
    df_overnight_heat_filtered = df_overnight_heat[pd.to_numeric(df_overnight_heat['연도'], errors='coerce').notna()].copy()
    df_overnight_heat_filtered['년'] = pd.to_numeric(df_overnight_heat_filtered['연도'])

    df_heatwave_filtered = df_heatwave[pd.to_numeric(df_heatwave['연도'], errors='coerce').notna()].copy()
    df_heatwave_filtered['년'] = pd.to_numeric(df_heatwave_filtered['연도'])

    df_coldwave_filtered = df_coldwave[pd.to_numeric(df_coldwave['연도'], errors='coerce').notna()].copy()
    df_coldwave_filtered['년'] = pd.to_numeric(df_coldwave_filtered['연도'])

    #연도 별 월 별을 각 행으로 배치하여 각 인덱스에 일수 삽입
    df_overnight_heat_melted = df_overnight_heat_filtered.melt(id_vars='년', value_vars=['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월'], var_name='월', value_name='일수')
    df_overnight_heat_melted['기온'] = '열대야일수'

    df_heatwave_melted = df_heatwave_filtered.melt(id_vars='년', value_vars=['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월'], var_name='월', value_name='일수')
    df_heatwave_melted['기온'] = '폭염일수'

    df_coldwave_melted = df_coldwave_filtered.melt(id_vars='년', value_vars=['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월'], var_name='월', value_name='일수')
    df_coldwave_melted['기온'] = '한파일수'

    #년,월을 수치형 변환
    df_overnight_heat_melted['년'] = pd.to_numeric(df_overnight_heat_melted['년'])
    df_overnight_heat_melted['월'] = df_overnight_heat_melted['월'].str.replace('월', '').astype(int)

    df_heatwave_melted['년'] = pd.to_numeric(df_heatwave_melted['년'])
    df_heatwave_melted['월'] = df_heatwave_melted['월'].str.replace('월', '').astype(int)

    df_coldwave_melted['년'] = pd.to_numeric(df_coldwave_melted['년'])
    df_coldwave_melted['월'] = df_coldwave_melted['월'].str.replace('월', '').astype(int)

    #하나의 파일로 만들기
    df_concat = pd.concat([df_overnight_heat_melted, df_heatwave_melted, df_coldwave_melted])

    df_pivot = df_concat.pivot(index=['년', '월'], columns='기온', values='일수').reset_index()
    df_pivot[df_pivot['년']!=2025]
    #식별자를 위해 '시도' 를 i->도시이름 작성
    df_pivot['시도']= city_col[num]
    #지역별 csv파일로 변환
    df_pivot.to_csv(f'{city[num]}_day.csv', encoding='euc-kr', index=False)
    num+=1
    print(df_pivot)

number = 0
for i in city:
    if number == 0:
        df= pd.read_csv(f'{i}.csv',encoding='cp949') #1번 파일 불러오기
        df_day= pd.read_csv(f'{i}_day.csv',encoding='cp949') #2번 파일 불러오기
        df_merge = pd.merge(df,df_day,on=['년','월','시도'],how='left') # 년과 월을 기준으로 Left join
        combine_df=df_merge
        number+=1
    else:
        df= pd.read_csv(f'{i}.csv',encoding='cp949')
        df_day= pd.read_csv(f'{i}_day.csv',encoding='cp949')
        df_merge = pd.merge(df,df_day,on=['년','월','시도'],how='left')
        combine_df=pd.concat([combine_df,df_merge]) #모든 데이터를 concat

combine_df.to_csv('combine_df.csv', encoding='euc-kr', index=False)

Data encoding

In [ ]:
# Detect encoding
def detect_encoding(file: str):
	with open(file, 'rb') as f:
		raw_data = f.read(10000)
		result = chardet.detect(raw_data)
		print(result)

Extreme weather

In [31]:
# Load file
df_extreme_weather = pd.read_csv('data/cleaned/weather/extreme_weather.csv', encoding='euc-kr')

# Rename headers
df_extreme_weather.columns = ["Year",
							  "Month",
							  "Number of Tropical Nights",
							  "Number of Heat Wave Days",
							  "Number of Cold Wave Days",
							  "Region"]

# Translate Korean to English
kor_to_eng = {"서울특별시": "Seoul",
    		  "부산광역시": "Busan",
    		  "대구광역시": "Daegu",
    		  "인천광역시": "Incheon",
    		  "광주광역시": "Gwangju",
    		  "대전광역시": "Daejeon",
    		  "울산광역시": "Ulsan"}
df_extreme_weather['Region'] = df_extreme_weather['Region'].map(kor_to_eng)

# Save file
df_extreme_weather.to_csv('data/cleaned/weather/extreme_weather_cleaned.csv', index=False, encoding='utf-8')

Weather forecast

In [32]:
# Weather forecast
# Load file
df_weather_forecast = pd.read_csv('data/cleaned/weather/weather_forecast.csv', encoding='euc-kr')
df_weather_forecast = df_weather_forecast.drop(columns=['지점', '최고기온(°C)', '최저기온(°C)', '최고기온 나타난날(yyyymmdd)','최저기온 나타난날(yyyymmdd)',  
														'최고해면기압(hPa)', '최저해면기압(hPa)', '최고해면기압 나타난날(yyyymmdd)', '최저해면기압 나타난날(yyyymmdd)', 
														'최고수증기압(hPa)', '최저수증기압(hPa)', '최고수증기압 나타난날(yyyymmdd)', '최저수증기압 나타난날(yyyymmdd)',
														'최소상대습도(%)', '최소상대습도 나타난날(yyyymmdd)', 
														'일최다강수량(mm)', '1시간최다강수량(mm)', '10분최다강수량(mm)', '일최다강수량 나타난날(yyyymmdd)', '1시간최다강수량 나타난날(yyyymmdd)', '10분최다강수량 나타난날(yyyymmdd)', 
														'소형일최대증발량(mm)', '대형총증발량(mm)', '대형일최대증발량(mm)', '소형일최대증발량 나타난날(yyyymmdd)', '대형일최대증발량 나타난날(yyyymmdd)', 
														'최대풍속(m/s)', '최대순간풍속(m/s)', '최대풍속 풍향(16방위)', '최대순간풍속 풍향(16방위)', '최대풍속 나타난날(yyyymmdd)', '최대순간풍속 나타난날(yyyymmdd)', '최다풍향(16방위)', 
														'평균중하층운량(1/10)', '합계 일조시간(hr)', 
														'최심적설(cm)', '최심신적설(cm)', '3시간신적설합(cm)', '최심적설 나타난날(yyyymmdd)', '최심신적설 나타난날(yyyymmdd)', 
														'최저초상온도(°C)', '최저초상온도 나타난날(yyyymmdd)', 
														'0.05m평균지중온도(°C)', '0.1m평균지중온도(°C)', '0.2m평균지중온도(°C)', '0.3m평균지중온도(°C)', '0.5m평균지중온도(°C)', '1.0m평균지중온도(°C)', '1.5m평균지중온도(°C)', '3.0m평균지중온도(°C)', '5.0m평균지중온도(°C)'])

# Rename headers
df_weather_forecast.columns = ["Region",
							   "Date",
							   "Avg Temperature (Celsius)",
							   "Avg Max Temperature (Celsius)",
							   "Avg Min Temperature (Celsius)",
							   "Avg Local Pressure (hPa)",
							   "Avg Sea Level Pressure (hPa)",
							   "Avg Vapor Pressure (hPa)",
							   "Avg Dew Point Temp (Celsius)",
							   "Avg Relative Humidity (%)",
							   "Monthly Precipitation (mm)",
							   "Small Pan Evaporation (mm)",
							   "Avg Wind Speed (m/s)",
							   "Avg Cloud Cover (1/10)",
							   "Sunshine Rate (%)",
							   "Total Solar Radiation (MJ/m^2)",
							   "Avg Min Supercool Temp (Celsius)",
							   "Avg Ground Temp (Celsius)"]

# Translate Korean to English
kor_to_eng = {"서울": "Seoul",
    		  "부산": "Busan",
    		  "대구": "Daegu",
    		  "인천": "Incheon",
    		  "광주": "Gwangju",
    		  "대전": "Daejeon",
    		  "울산": "Ulsan"}
df_weather_forecast['Region'] = df_weather_forecast['Region'].map(kor_to_eng)

# Convert the Date to datetime format and extract year and month
df_weather_forecast['Date'] = pd.to_datetime(df_weather_forecast['Date'])
df_weather_forecast['Year'] = df_weather_forecast['Date'].dt.year
df_weather_forecast['Month'] = df_weather_forecast['Date'].dt.month
df_weather_forecast = df_weather_forecast.drop(columns=['Date'])

# Save file
df_weather_forecast.to_csv('data/cleaned/weather/weather_forecast_cleaned.csv', index=False, encoding='utf-8')

Electrcity consumption

In [33]:
# Load file
df_electricity = pd.read_csv('data/cleaned/electricity/electricity_consumption.csv', encoding='euc-kr')
df_electricity = df_electricity.drop(columns=['시군구', '가구당 평균 전기요금(원)'])

# Rename headers
df_electricity.columns = ["Year",
						  "Month",
						  "Region",
						  "Number of Households",
						  "Avg Power Consumption per Household (kWh)"]

# Translate Korean to English
kor_to_eng = {"서울특별시": "Seoul",
    		  "부산광역시": "Busan",
    		  "대구광역시": "Daegu",
    		  "인천광역시": "Incheon",
    		  "광주광역시": "Gwangju",
    		  "대전광역시": "Daejeon",
    		  "울산광역시": "Ulsan"}
df_electricity['Region'] = df_electricity['Region'].map(kor_to_eng)

# Save file
df_electricity.to_csv('data/cleaned/electricity/electricity_consumption_cleaned.csv', index=False, encoding='utf-8')

Gas supply

In [ ]:
# Load file
df_gas = pd.read_csv("data/cleaned/gas/gas_supply.csv", encoding='euc-kr')

# Rename headers
df_gas.columns = ['Date',
						 'Seoul',
						 'Incheon',
						 'Gyeonggi',
						 'Gangwon',
						 'Chungcheong',
						 'Jeonbuk',
						 'Gwangju',
						 'Daegu',
						 'Busan',
						 'Jeju']

# Convert the Date to datetime format and extract year and month
df_gas['Date'] = pd.to_datetime(df_gas['Date'])
df_gas['Year'] = df_gas['Date'].dt.year
df_gas['Month'] = df_gas['Date'].dt.month
df_gas = df_gas.drop(columns=['Date'])

# Drop unnecessary columns
df_gas = df_gas.drop(columns=['Gyeonggi', 'Gangwon', 'Chungcheong', 'Jeonbuk', 'Jeju'])

# Reshape
df_gas = pd.melt(df_gas,
				 id_vars=['Year', 'Month'],
				 value_vars=['Seoul', 'Incheon', 'Gwangju', 'Daegu', 'Busan'],
				 var_name='Region',
				 value_name='Gas Supply (ton)')

# Extrapolate Ulsan based on mean values
monthly_avg = df_gas.groupby(['Year', 'Month'])['Gas Supply (ton)'].mean().reset_index()
monthly_avg['Region'] = 'Ulsan'
df_ulsan = monthly_avg[['Year', 'Month', 'Region', 'Gas Supply (ton)']]
df_gas = pd.concat([df_gas, df_ulsan], ignore_index=True)

# Save csv
df_gas.to_csv('data/cleaned/gas/gas_supply_cleaned.csv', index=False, encoding='utf-8')


Population

In [35]:
# Load file
df_population = pd.read_csv('data/cleaned/population/population.csv', encoding='euc-kr')

# Rename first column to 'Region'
df_population = df_population.rename(columns={df_population.columns[0]: 'Region'})

# Translate Korean to English
kor_to_eng = {'계 (천 명)': 'Total (k)',
			  '전국': 'National',
			  '서울': 'Seoul',
			  '부산': 'Busan',
			  '대구': 'Daegu',
			  '인천': 'Incheon',
			  '광주': 'Gwangju',
			  '대전': 'Daejeon',
			  '울산': 'Ulsan',
			  '세종': 'Sejong',
			  '경기': 'Gyeonggi',
			  '강원': 'Gangwon',
			  '충북': 'Chungbuk',
			  '충남': 'Chungnam',
			  '전북': 'Jeonbuk',
			  '전남': 'Jeonnam',
			  '경북': 'Gyeongbuk',
			  '경남': 'Gyeongnam',
			  '제주': 'Jeju',
			  '수도권': 'Capital Area'}
df_population['Region'] = df_population['Region'].replace(kor_to_eng)

# Melt the wide format → long format
df_population = df_population.melt(id_vars='Region', var_name='Year', value_name='Population')

# Clean numeric values (remove commas)
df_population['Population'] = df_population['Population'].replace(',', '', regex=True)

# Convert to appropriate types
df_population['Year'] = df_population['Year'].astype(int)
df_population['Population'] = df_population['Population'].astype(float)

# Save file
df_population.to_csv('data/cleaned/population/population_cleaned.csv', index=False, encoding='utf-8')

Merge data

In [38]:
# Merge data
df = pd.merge(df_electricity, df_gas, on=['Region', 'Year', 'Month'], how='inner')
df = pd.merge(df, df_extreme_weather, on=['Region', 'Year', 'Month'], how='inner')
df = pd.merge(df, df_weather_forecast, on=['Region', 'Year', 'Month'], how='inner')

# Compute average gas supply per household
df['Gas Supply per Household (ton)'] = df['Gas Supply (ton)'] * 0.437 / df['Number of Households']
df = df.drop(columns=['Gas Supply (ton)'])

# Save file
df.to_csv('data/cleaned/data_merged.csv', index=False, encoding='utf-8')